In [20]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, Dropout, SimpleRNN
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.regularizers import l2

import numpy as np
import pandas as pd
import re


In [21]:
# Hyperparams
vocab_size = 10000
sequence_length = 100
embedding_size = 256
batch_size = 64
epochs = 5
TRAIN_TEST_SPLIT = 0.8

In [22]:
data = pd.read_csv('IMDB Dataset.csv')
data.replace({'positive': 1, 'negative': 0}, inplace=True)
data.head()

C:\Users\JS\AppData\Local\Temp\ipykernel_10052\3112354220.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'positive': 1, 'negative': 0}, inplace=True)


,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [23]:
print(data['review'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fac

In [24]:
#removing <br > function
def remove_html(row):
    result = re.sub(r"<br />", "", row['review'])
    return result

In [25]:
data['review'] = data.apply(remove_html, axis=1)
data.head()['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wo

In [26]:
d = data.to_numpy()
texts = d[:,0]
X_train = texts[:int(len(texts)*TRAIN_TEST_SPLIT)]
X_test = texts[int(len(texts)*TRAIN_TEST_SPLIT):]
labels = d[:,1]
y_train = labels[:int(len(texts)*TRAIN_TEST_SPLIT)]
y_test = labels[int(len(texts)*TRAIN_TEST_SPLIT):]

In [27]:
vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length,
    name='textvectorizer'
)

In [28]:
vectorize_layer.adapt(X_train)

In [29]:
vectorized_train = vectorize_layer(X_train)
vectorized_test = vectorize_layer(X_test)

In [30]:
# Step 2: Model Definition
model = Sequential([
    Embedding(vocab_size, embedding_size, name="embedding"),
    SimpleRNN(20, name="rnn"),
    Dropout(0.3),
    Dense(1,activation="sigmoid", name="output")
])

In [31]:
# Step 3: Compile the Model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), metrics=['accuracy'])


In [32]:
y_train = np.array(y_train,dtype='float32')
y_test = np.array(y_test,dtype='float32')

In [33]:
# Step 4: Train the Model
model.fit(vectorized_train, y_train, epochs=5, verbose=1, batch_size=batch_size)

Epoch 1/5
625/625 [==============================] - 22s 34ms/step - loss: 0.5929 - accuracy: 0.6950
Epoch 2/5
625/625 [==============================] - 20s 33ms/step - loss: 0.4801 - accuracy: 0.7893
Epoch 3/5
625/625 [==============================] - 21s 33ms/step - loss: 0.5286 - accuracy: 0.7247
Epoch 4/5
625/625 [==============================] - 20s 32ms/step - loss: 0.5560 - accuracy: 0.7267
Epoch 5/5
625/625 [==============================] - 20s 32ms/step - loss: 0.4443 - accuracy: 0.7994


In [34]:
model.evaluate(vectorized_test, y_test, verbose=1)

313/313 [==============================] - 2s 5ms/step - loss: 0.6466 - accuracy: 0.7178


[0.6465654373168945, 0.7178000211715698]

In [35]:
print(data['review'][0])

One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where other shows wou

In [36]:
# Make a prediction on a single review
vectorized_example = vectorize_layer(tf.convert_to_tensor([data['review'][0]]))  # Convert to tensor and add batch dimension
print("model prediction: ", model.predict(vectorized_example, verbose=0), "true value: ", data['sentiment'][0])

model prediction:  [[0.90803456]] true value:  1


In [37]:
for i in range(10):
    print(i,"review: ",data['review'][i])
    vectorized_example = vectorize_layer(tf.convert_to_tensor([data['review'][i]]))
    print("model prediction: ", model.predict(vectorized_example, verbose=0))
    print("true value: ",data['sentiment'][i])

0 review:  One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.I would say the main appeal of the show is due to the fact that it goes where othe

In [38]:
custom_reviews = ["This movie is absolutely horrible I couldn't sit through all of it", "I love this movie, it's the greatest thing since the invention of sliced bread!","I would rather to swallow a gallon gasoline rather than watch 5 more minutes of this horrible trash"]
custom_labels = [0,1,0]

for i in range(len(custom_labels)):
    print(f"{i} review: {custom_reviews[i]}")
    
    vectorized_example = vectorize_layer(tf.convert_to_tensor([custom_reviews[i]]))
    
    prediction = model.predict(vectorized_example, verbose=0)
    print("model prediction: ", prediction[0][0])
    print("true value: ", custom_labels[i], "\n")

0 review: This movie is absolutely horrible I couldn't sit through all of it
model prediction:  0.084422946
true value:  0 

1 review: I love this movie, it's the greatest thing since the invention of sliced bread!
model prediction:  0.75885
true value:  1 

2 review: I would rather to swallow a gallon gasoline rather than watch 5 more minutes of this horrible trash
model prediction:  0.08442303
true value:  0 



# Source:
https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

inspired by https://www.youtube.com/watch?v=FTCpdNuIiZM&t=4s